In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt

In [2]:
# Load Cifer-10
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()
X_train, X_test= X_train/255.0, X_test/255.0

In [4]:
# subset for tuning (10k samples)
X_tune= X_train[:10000]
y_tune= y_train[:10000]

In [27]:
def build_model(hp):
    inputs = keras.Input(shape=(32,32,3))
    x=inputs

    #convolutional blocks
    for i in range(hp.Int("num_conv_blocks", 1, 3)):
        x= layers.Conv2D(
            filters=hp.Int(f" filters_{i}", 32, 128, step=32),
            kernel_size = 3,
            padding="same",
            activation="relu")(x)
        x=layers.MaxPooling2D()(x)
    x = layers.Flatten()(x)

    x= layers.Dense(
        units=hp.Int("dense_units", 64, 256, step=64),
        activation='relu'
    )(x)
    outputs = layers.Dense(10, activation="softmax")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(optimizer = keras.optimizers.Adam(
        hp.Choice("lr", [1e-2, 1e-3, 1e-4])
    ), 
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
    )
    return model

In [28]:
tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy", 
    max_trials=5, 
    executions_per_trial=1,
    directory= "cnn_tuner_results", 
    project_name="cifar10_cnn2"
)

Reloading Tuner from cnn_tuner_results\cifar10_cnn2\tuner0.json


In [29]:
tuner.search(X_tune, y_tune, epochs=5, validation_split=0.2)

In [30]:
best_hps = tuner.get_best_hyperparameters(1)[0]

print("Best Hyperparameters:")
print(f"Conv Blocks: {best_hps.get('num_conv_blocks')}")
for i in range(best_hps.get('num_conv_blocks')):
    print(f"Filters in Block {i+1}: {best_hps.get(f' filters_{i}')}")  # ✅ fixed key name
print(f"Dense Units: {best_hps.get('dense_units')}")
print(f"Learning Rate: {best_hps.get('lr')}")


Best Hyperparameters:
Conv Blocks: 2
Filters in Block 1: 32
Filters in Block 2: 32
Dense Units: 256
Learning Rate: 0.001


In [ ]:
final_model = tuner.hypermodel.build(best_hps)
final_model.fit(X_train, y_train, epochs= 10, validation_split=0.1)

Epoch 1/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 42s 28ms/step - accuracy: 0.4057 - loss: 1.6501 - val_accuracy: 0.5918 - val_loss: 1.1588
Epoch 2/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 39s 28ms/step - accuracy: 0.6209 - loss: 1.0818 - val_accuracy: 0.6476 - val_loss: 1.0026
Epoch 3/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 41s 28ms/step - accuracy: 0.6807 - loss: 0.9078 - val_accuracy: 0.6822 - val_loss: 0.9191
Epoch 4/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 40s 29ms/step - accuracy: 0.7297 - loss: 0.7832 - val_accuracy: 0.6968 - val_loss: 0.8703
Epoch 5/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 41s 29ms/step - accuracy: 0.7709 - loss: 0.6588 - val_accuracy: 0.6922 - val_loss: 0.9270
Epoch 6/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 82s 30ms/step - accuracy: 0.7975 - loss: 0.5759 - val_accuracy: 0.7042 - val_loss: 0.8954
Epoch 7/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 81s 29ms/step - accuracy: 0.8344 - loss: 0.4790 - val_accuracy: 0.7150 - val_loss: 0.8954
Epoch 8/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 41s 29ms/step - accuracy: 0.8672 -